In [9]:
import tensorflow as tf
import keras
from keras.models import load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import pandas
import h5py
import glob

In [92]:
h5file =  "VGG19_weights.h5"

with h5py.File(h5file,'r') as fid:
     model = tf.keras.models.load_model(fid)

In [11]:
def get_filenames():
    global path
    path = r"test"
    return os.listdir(path)

In [12]:
def autoroi(img):

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    thresh = cv2.threshold(gray_img, 130, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=5)

    contours, hierarchy = cv2.findContours(
        thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    biggest = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(biggest)
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 256, 0), 2)
    roi = img[y:y+h, x:x+w]

    return roi

In [97]:
def prediction():
    list_of_files = glob.glob('Images/*') #testing different files
#     latest_file = max(list_of_files, key=os.path.getctime)
    total = len(list_of_files)
    print(total)
    count = 0
    for latest_file in list_of_files:
        
        img = cv2.imread(latest_file)
        img = autoroi(img)
        img = cv2.resize(img, (224, 224))
        img = np.reshape(img, [1, 224, 224, 3])
    #     img = tf.cast(img, tf.float64)


        prediction = model.predict(img)
        print(prediction)
        prediction = prediction.argmax()
        print(prediction)
        if prediction == 1:
            count += 1

    print(count)
    return(prediction)

In [98]:
finalPrediction = prediction()

705
[[1. 0.]]
0
[[5.385852e-25 1.000000e+00]]
1
[[3.7016865e-22 1.0000000e+00]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[1. 0.]]
0
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[3.6900825e-30 1.0000000e+00]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[8.555869e-31 1.000000e+00]]
1
[[0. 1.]]
1
[[3.9822648e-16 1.0000000e+00]]
1
[[1.000000e+00 1.409736e-12]]
0
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[1. 0.]]
0
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[1.3595905e-35 1.0000000e+00]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[1.000000e+00 7.877675e-24]]
0
[[1.0000000e+00 9.3098976e-32]]
0
[[0. 1.]]
1
[[1. 0.]]
0
[[0. 1.]]
1
[[0. 1.]]
1
[[9.9999917e-01 8.5378048e-07]]
0


ValueError: max() arg is an empty sequence